# Emoji Predictor Project
----------

### let's get the modules

In [1]:
import numpy as np
import pandas as pd 
import tensorflow as tf
import keras as kr

In [7]:
# Importing datasets
train_data=pd.read_csv(r'C:\Users\bhavy\OneDrive\Documents\csv files\emojiprediction\emoji.csv',header=None)
test_data=pd.read_csv(r'C:\Users\bhavy\OneDrive\Documents\csv files\emojiprediction\emoji_test.csv',header=None)

### Analyzing our data

In [8]:
train_data.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [9]:
test_data.head()

,0,1
0,I want to eat\t,4
1,he did not answer\t,3
2,he got a raise\t,2
3,she got me a present\t,0
4,ha ha ha it was so funny\t,2


In [11]:
train_data[1].unique()

array([3, 2, 0, 4, 1], dtype=int64)

these values represents classes where each of these will donate one single emoji

# Using Emojize module

In [16]:
!pip install emoji
import emoji     # this module is used for visualizing our emojis

In [14]:
# there are three ways to visualize emojis
# i) using emojize library
emoji.emojize(':baseball:')

'⚾'

In [17]:
emoji.emojize(':beaming_face_with_smiling_eyes:')

'😁'

In [19]:
# ii) using CLDR name
print("\N{Grinning Face}")

😀


In [20]:
# iii) using unicodes
print('\U0001f600')

😀


now we will create a dictionary to tell the system which value represents which emoji

In [21]:
# creating a dictionary for getting emojis
emoji_dictionary={"0": "\u2764\uFE0F",
                 "1":":baseball:",
                 "2":":beaming_face_with_smiling_eyes:",
                 "3":":downcast_face_with_sweat:",
                "4":":fork_and_knife:"}

In [23]:
for i in emoji_dictionary.values():
    print(emoji.emojize(i))

❤️
⚾
😁
😓
🍴


In [24]:
# getting training data from training dataset
train_x=train_data[0]
train_y=train_data[1]

In [25]:
# getting test data
test_x=test_data[0]
test_y=test_data[1]

In [28]:
# now converting target values(i.e value to be predicted) to categorical values
train_y=kr.utils.to_categorical(train_y)
test_y=kr.utils.to_categorical(test_y)

In [29]:
train_y[0]

array([0., 0., 0., 1., 0.], dtype=float32)

In [30]:
# now getting file named 'glove' to convert words to machine understandable form
embeddings={}
with open(r'C:\Users\bhavy\OneDrive\Documents\glove.6B.50d.txt',encoding="UTF-8") as f:
    for line in f:
        values=line.split()
        word=values[0]
        coeffs=np.asarray(values[1:],dtype='float32')
        embeddings[word]=coeffs
        

 link for downloading glove file: https://www.kaggle.com/watts2/glove6b50dtxt

In [31]:
embeddings['the'].shape

(50,)

In [32]:
# Creating a function to get numeric values for each word
def get_emb(X):
    matrix=np.zeros((X.shape[0],10,50))
    for ix in range(X.shape[0]):
        X[ix]=X[ix].split()
        for jx in range(len(X[ix])):
            matrix[ix][jx]=embeddings[X[ix][jx].lower()]
    return matrix

In [33]:
# Stroring the numerical values of features into emb_train and emb_test
emb_train_x=get_emb(train_x)
emb_test_x=get_emb(test_x)

C:\Users\bhavy\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


# Creating a Recurrent Neural Network

In [36]:
# Creating a recurrent neural network
from keras.layers import *
model=kr.models.Sequential()
model.add(SimpleRNN(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.4))      # DEACTIVATES CERTAIN PERCENTAGE OF NEURONS
model.add(SimpleRNN(64))
model.add(Dropout(0.3))      # USED FOR CHECKING UNDER FITTING AND OVER FITTING
model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_3 (SimpleRNN)     (None, 10, 64)            7360      
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, 64)                8256      
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
Total params: 15,941
Trainable params: 15,941
Non-trainable params: 0
__________________________________________________

In [37]:
# Compiling model
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])

In [38]:
# Training our neural network with training data
batchsize=32
epochs=40
model.fit(emb_train_x,train_y,batch_size=batchsize,epochs=epochs,validation_split=0.1)


Train on 118 samples, validate on 14 samples
Epoch 1/40
118/118 [==============================] - 2s 16ms/step - loss: 1.8343 - acc: 0.2542 - val_loss: 1.7095 - val_acc: 0.1429
Epoch 2/40
118/118 [==============================] - 0s 576us/step - loss: 1.5507 - acc: 0.2966 - val_loss: 1.6873 - val_acc: 0.1429
Epoch 3/40
118/118 [==============================] - 0s 509us/step - loss: 1.5503 - acc: 0.3136 - val_loss: 1.6803 - val_acc: 0.2143
Epoch 4/40
118/118 [==============================] - 0s 509us/step - loss: 1.3407 - acc: 0.4068 - val_loss: 1.5855 - val_acc: 0.3571
Epoch 5/40
118/118 [==============================] - 0s 441us/step - loss: 1.2730 - acc: 0.4407 - val_loss: 1.5321 - val_acc: 0.4286
Epoch 6/40
118/118 [==============================] - 0s 441us/step - loss: 1.3122 - acc: 0.4492 - val_loss: 1.5008 - val_acc: 0.4286
Epoch 7/40
118/118 [==============================] - 0s 475us/step - loss: 1.1504 - acc: 0.5593 - val_loss: 1.4603 - val_acc: 0.4286
Epoch 8/40
118/11

In [39]:
# Evaluating our model on test data
model.evaluate(emb_test_x,test_y)

56/56 [==============================] - 0s 429us/step


[1.557425584111895, 0.5357142686843872]

# Predicting emojis

In [40]:
# predicting emojis using our rnn model
pred=model.predict_classes(emb_test_x)

In [42]:
# Displaying result
for i in range(30):
    print(' '.join(test_x[i]),'  ',emoji.emojize(emoji_dictionary[str(pred[i])]))
    print()

I want to eat    🍴

he did not answer    😓

he got a raise    😓

she got me a present    😓

ha ha ha it was so funny    😁

he is a good friend    😁

I am upset    😓

We had such a lovely dinner tonight    😁

where is the food    🍴

Stop making this joke ha ha ha    😁

where is the ball    ⚾

work is hard    😓

This girl is messing with me    😁

are you serious ha ha    😓

Let us go play baseball    ⚾

This stupid grader is not working    😓

work is horrible    😓

Congratulation for having a baby    😁

stop messing around    🍴

any suggestions for dinner    🍴

I love taking breaks    😓

you brighten my day    ❤️

I boiled rice    🍴

she is a bully    😁

Why are you feeling bad    😓

I am upset    😓

I worked during my birthday    ⚾

My grandmother is the love of my life    😁

enjoy your break    ⚾

valentine day is near    😁

